در این قسمت میخواهیم موارد زیر را تعریف کنیم <br/>
<h3>action</h3> در هر مرحله پنج تا داریم که یعنی بالا رفتن، پایین، چپ، راست و توپ برداشتن <br/>
<h3>state</h3> استیت را اینگونه تعریف کردم که مختصات فعلی، توپ هایی که در حال حمل هستند و توپ هایی که هنوز برداشته نشدن <br/>
<h3>initial state</h3> همه توپ ها سر جایشان هستند، هیچ توپی در حال حمل نیست و مختصات فعلی همان مختصات صورت سوال است <br/>
<h3>goal state</h3> هیچ توپی توی نقشه نیست که برداشته نشده باشد، و هیچ توپی در حال حمل نیست <br/>

<strong>in this computer assignment we are going to implement and find the differences between uninformed search algorithms (BFS and IDS) and informed search algorithms (A* and weighteA*).</strong>


<h1>BFS</h1>

In [17]:
import time
class Node:
    def __init__(self,state,parent = None,actions = 0, ballsTaken = 0):
        self.state = state
        self.actions = actions
        self.ballsTaken = ballsTaken
        self.parent = parent
    
    def __repr__(self):
        return str(self.state.person)
                
class State:
    def __init__(self,person,balls = [],ballsTarget = [],ballsCarryTarget = []):
        self.person = person
        self.balls = balls
        self.ballsTarget = ballsTarget
        self.ballsCarryTarget = ballsCarryTarget
    
    def giveMeTuple(self):
        return (tuple(self.person),tuple(self.balls),tuple(self.ballsCarryTarget))
        
    def __eq__(self,other):
        return self.person == other.person and self.balls == other.balls and self.ballsCarryTarget == other.ballsCarryTarget
    

def getInput():
    file = open('Tests/4.txt','r')
    lines = file.readlines()
    file.close()
    rowcol = list(map(int,lines[0].split()))
    startCords = list(map(int,lines[1].split()))
    endCords = list(map(int,lines[2].split()))
    bagCapacity = int(lines[3])
    numOfBalls = int(lines[4])
    startBallsCords = []
    endBallCords = []
    for n in range(numOfBalls):
        startEndCordBall = list(map(int,lines[n + 5].split()))
        startBallsCords.append(tuple(startEndCordBall[:2]))
        endBallCords.append(tuple(startEndCordBall[2:]))
    maze = []
    for i in range(rowcol[0]):
        maze.append(lines[5 + numOfBalls  + i].split())
    return maze, startCords, endCords, bagCapacity, startBallsCords, endBallCords

def addToFrontierExplored(state,explored,frontier,currNode,allStates,plusWhat = 0):
    allStates[0] += 1
    if state.giveMeTuple() not in explored:
        node = Node(state, currNode,currNode.actions + 1,currNode.ballsTaken + plusWhat)
        frontier.append(node)
        explored[state.giveMeTuple()] = ""    

def moveToRight(maze,currNode,explored,frontier,allStates):
    if currNode.state.person[1] < len(maze[0]) - 1:
        if maze[currNode.state.person[0]][currNode.state.person[1] + 1] != '*':
            state = State([currNode.state.person[0] , currNode.state.person[1] + 1],currNode.state.balls[:],currNode.state.ballsTarget[:],currNode.state.ballsCarryTarget[:])
            addToFrontierExplored(state,explored,frontier,currNode,allStates)

def moveToLeft(maze,currNode,explored,frontier,allStates):
    if currNode.state.person[1] > 0:
        if maze[currNode.state.person[0]][currNode.state.person[1] - 1] != '*': 
            state = State([currNode.state.person[0] , currNode.state.person[1] - 1],currNode.state.balls[:],currNode.state.ballsTarget[:],currNode.state.ballsCarryTarget[:])
            addToFrontierExplored(state,explored,frontier,currNode,allStates)


def moveToUp(maze,currNode,explored,frontier,allStates):
    if currNode.state.person[0] > 0:
        if maze[currNode.state.person[0] - 1][currNode.state.person[1]] != '*':
            state = State([currNode.state.person[0] - 1 , currNode.state.person[1]],currNode.state.balls[:],currNode.state.ballsTarget[:],currNode.state.ballsCarryTarget[:])
            addToFrontierExplored(state,explored,frontier,currNode,allStates)

def moveToDown(maze,currNode,explored,frontier,allStates):
    if currNode.state.person[0] < len(maze) - 1:
        if maze[currNode.state.person[0] + 1][currNode.state.person[1]] != '*':
            state = State([currNode.state.person[0] + 1 , currNode.state.person[1]],currNode.state.balls[:],currNode.state.ballsTarget[:],currNode.state.ballsCarryTarget[:])
            addToFrontierExplored(state,explored,frontier,currNode,allStates)

def takeBall(currNode,bagCapacity,explored,frontier,allStates):
    if currNode.ballsTaken < bagCapacity:
        for ballIndex in range(len(currNode.state.balls)):
            if tuple(currNode.state.person) == currNode.state.balls[ballIndex]:
                state = State(currNode.state.person,currNode.state.balls[:],currNode.state.ballsTarget[:],currNode.state.ballsCarryTarget[:])
                state.ballsCarryTarget.append(state.ballsTarget.pop(ballIndex))
                state.balls.pop(ballIndex)
                addToFrontierExplored(state,explored,frontier,currNode,allStates,1)
                break

def putDownBall(currNode):
    for targetBall in range(len(currNode.state.ballsCarryTarget)):
        if tuple(currNode.state.person) == currNode.state.ballsCarryTarget[targetBall]:
            currNode.state.ballsCarryTarget.pop(targetBall)
            currNode.ballsTaken -= 1
            break

def findPath(lastNode):
    result = []
    while lastNode != None:
        result.append(lastNode.state.person)
        lastNode = lastNode.parent
    return str(result[::-1])

def BFS(frontier,explored,maze,allStates):
    while 1:
        if len(frontier) == 0: 
            print(-1)
            return None
        currNode = frontier.pop(0)
        putDownBall(currNode)
        if currNode.state == goalState:
            print("requires {} moves".format(currNode.actions))
            return currNode

        moveToUp(maze,currNode,explored,frontier,allStates)
        moveToDown(maze,currNode,explored,frontier,allStates)
        moveToLeft(maze,currNode,explored,frontier,allStates)
        moveToRight(maze,currNode,explored,frontier,allStates)
        takeBall(currNode,bagCapacity,explored,frontier,allStates)
        

if __name__ == "__main__":
    maze, initialCord, goalCord, bagCapacity, balls, ballsTarget = getInput()
    initialState = State(initialCord,balls,ballsTarget)
    initialNode = Node(initialState)
    goalState = State(goalCord)
    frontier = [initialNode]
    explored = {initialState.giveMeTuple() : ""}
    allStates = [0]

    t1 = time.time()
    lastNode = BFS(frontier,explored,maze,allStates)
    t2 = time.time()

    if lastNode != None: print("path is {}".format(findPath(lastNode)))
    print("distinct states visited are {}".format(len(explored)))
    print("all states {}".format(allStates[0]))
    print("time = {}".format(t2 - t1))



requires 92 moves
path is [[1, 0], [1, 1], [1, 2], [1, 3], [2, 3], [3, 3], [4, 3], [5, 3], [5, 4], [5, 5], [5, 6], [5, 7], [5, 8], [5, 9], [5, 10], [6, 10], [7, 10], [8, 10], [9, 10], [10, 10], [11, 10], [12, 10], [12, 9], [12, 8], [12, 7], [13, 7], [14, 7], [15, 7], [15, 6], [15, 5], [15, 4], [16, 4], [17, 4], [18, 4], [19, 4], [20, 4], [20, 5], [20, 6], [20, 7], [20, 8], [20, 9], [20, 10], [20, 11], [21, 11], [22, 11], [22, 12], [22, 13], [22, 14], [22, 15], [22, 16], [22, 17], [21, 17], [20, 17], [19, 17], [18, 17], [18, 18], [18, 19], [18, 20], [17, 20], [16, 20], [16, 21], [16, 22], [16, 23], [15, 23], [14, 23], [13, 23], [13, 24], [13, 25], [14, 25], [15, 25], [16, 25], [17, 25], [18, 25], [19, 25], [20, 25], [21, 25], [22, 25], [22, 24], [22, 23], [22, 22], [23, 22], [24, 22], [25, 22], [26, 22], [27, 22], [28, 22], [28, 23], [28, 24], [28, 25], [28, 26], [28, 27], [28, 28], [28, 29]]
distinct states visited are 457
all states 1019
time = 0.006012916564941406


<strong>as we can see for evey bigger test, the number of states that we see are increasing and it is much more than the number of states that we expand </strong>



<strong> when we implement IDS function we need an upper limit for the depth we are going to reach. and also we should use a stack instead of a queue to implement the DFS. when we reach the limit in a node and we haven't reached our gola state, we should skip expanding this node and not go any further than this state. but the conditions to expand each state is the same as BFS we implemented. our initial state is just like before, but the goal state here has 2 conditions. </strong>

1. if you have taken all the balls and you have reached the final coordinate.<br/>
2. if you have reached the limit and haven't found any final states and the stack is empty, return failure and increase the upper limit.<br/>

<strong> just like the bfs, IDS is complete and because the costs between two adjacent states are equal, here we have an optimal IDS.
but here we should check another condition. we may have seen a state with value C1 and we will see it again but with cost C2, if C2 is less than C1 it means that we can see a state which we have seen before but we see it with less cost so we should add it to the stack again. we check this condition using a dictionary instead of a list to check the explored. </strong>

<h1>IDS</h1>

In [26]:
import time
class Node:
    def __init__(self,state,parent = None,actions = 0, ballsTaken = 0):
        self.state = state
        self.state.actions = actions
        self.ballsTaken = ballsTaken
        self.parent = parent

    def __repr__(self):
        return str(self.state.person)
    
class State:
    actions = 0
    def __init__(self,person,balls = [],ballsTarget = [],ballsCarryTarget = []):
        self.person = person
        self.balls = balls
        self.ballsTarget = ballsTarget
        self.ballsCarryTarget = ballsCarryTarget
    
    def giveMeTuple(self):
        return (tuple(self.person),tuple(self.balls),tuple(self.ballsCarryTarget))

    def __eq__(self,prev):
        return self.person == prev.person and self.balls == prev.balls and self.ballsCarryTarget == prev.ballsCarryTarget

def getInput():
    file = open('Tests/4.txt','r')
    lines = file.readlines()
    file.close()
    rowcol = list(map(int,lines[0].split()))
    startCords = list(map(int,lines[1].split()))
    endCords = list(map(int,lines[2].split()))
    bagCapacity = int(lines[3])
    numOfBalls = int(lines[4])
    startBallsCords = []
    endBallCords = []
    for n in range(numOfBalls):
        startEndCordBall = list(map(int,lines[n + 5].split()))
        startBallsCords.append(tuple(startEndCordBall[:2]))
        endBallCords.append(tuple(startEndCordBall[2:]))
    maze = []
    for i in range(rowcol[0]):
        maze.append(lines[5 + numOfBalls  + i].split())
    return maze, startCords, endCords, bagCapacity, startBallsCords, endBallCords

def addToFrontierExplored(state,currNode,explored,frontier,allStates,plusWhat = 0):
    allStates[0] += 1
    if state.giveMeTuple() not in explored:
        node = Node(state,currNode,currNode.state.actions + 1,currNode.ballsTaken + plusWhat)
        frontier.append(node)
        explored[state.giveMeTuple()] = state.actions
    else:
        node = Node(state,currNode,currNode.state.actions + 1,currNode.ballsTaken + plusWhat)
        if explored[state.giveMeTuple()] > state.actions:
            explored[state.giveMeTuple()] = state.actions
            frontier.append(node)

def moveToRight(maze,currNode,explored,frontier,allStates):
    if currNode.state.person[1] < len(maze[0]) - 1:
        if maze[currNode.state.person[0]][currNode.state.person[1] + 1] != '*':
            state = State([currNode.state.person[0] , currNode.state.person[1] + 1],currNode.state.balls[:],currNode.state.ballsTarget[:],currNode.state.ballsCarryTarget[:])           
            addToFrontierExplored(state,currNode,explored,frontier,allStates)
            
def moveToLeft(maze,currNode,explored,frontier,allStates):
    if currNode.state.person[1] > 0:
        if maze[currNode.state.person[0]][currNode.state.person[1] - 1] != '*':
            state = State([currNode.state.person[0] , currNode.state.person[1] - 1],currNode.state.balls[:],currNode.state.ballsTarget[:],currNode.state.ballsCarryTarget[:])
            addToFrontierExplored(state,currNode,explored,frontier,allStates)

def moveToUp(maze,currNode,explored,frontier,allStates):
    if currNode.state.person[0] > 0:
        if maze[currNode.state.person[0] - 1][currNode.state.person[1]] != '*':
            state = State([currNode.state.person[0] - 1 , currNode.state.person[1]],currNode.state.balls[:],currNode.state.ballsTarget[:],currNode.state.ballsCarryTarget[:])
            addToFrontierExplored(state,currNode,explored,frontier,allStates)
                

def moveToDown(maze,currNode,explored,frontier,allStates):
    if currNode.state.person[0] < len(maze) - 1:
        if maze[currNode.state.person[0] + 1][currNode.state.person[1]] != '*':
            state = State([currNode.state.person[0] + 1 , currNode.state.person[1]],currNode.state.balls[:],currNode.state.ballsTarget[:],currNode.state.ballsCarryTarget[:])
            addToFrontierExplored(state,currNode,explored,frontier,allStates)

def takeBall(currNode,bagCapacity,explored,frontier,allStates):
    if currNode.ballsTaken < bagCapacity:
        for ballIndex in range(len(currNode.state.balls)):
            if tuple(currNode.state.person) == currNode.state.balls[ballIndex]:
                state = State(currNode.state.person,currNode.state.balls[:],currNode.state.ballsTarget[:],currNode.state.ballsCarryTarget[:])
                state.ballsCarryTarget.append(state.ballsTarget.pop(ballIndex))
                state.balls.pop(ballIndex)
                addToFrontierExplored(state,currNode,explored,frontier,allStates,1)
                break

def putDownBall(currNode):    
    for targetBall in range(len(currNode.state.ballsCarryTarget)):
        if tuple(currNode.state.person) == currNode.state.ballsCarryTarget[targetBall]:
            currNode.state.ballsCarryTarget.pop(targetBall)
            currNode.ballsTaken -= 1
            break

def dfsTillLimit(frontier,explored,maze,limit,allStates):
    repeat = True
    currNode = None
    while 1:
        if len(frontier) == 0:
            break

        currNode = frontier.pop(-1)

        if currNode.state.actions > limit:
            continue

        if currNode.ballsTaken > 0:
            putDownBall(currNode)

        if currNode.state == goalState:
            print("requires {} moves".format(currNode.state.actions))
            repeat = False
            return repeat, currNode
            

        moveToUp(maze,currNode,explored,frontier,allStates)
        moveToLeft(maze,currNode,explored,frontier,allStates)
        takeBall(currNode,bagCapacity,explored,frontier,allStates)
        moveToDown(maze,currNode,explored,frontier,allStates)
        moveToRight(maze,currNode,explored,frontier,allStates)

    return repeat,currNode

def IDS(initialCord,goalCord,maze,allStates):
    statesVisited = 0
    maxLimit = 10 ** 9
    # start = abs(goalCord[0] - initialCord[0]) + abs(goalCord[1] - initialCord[1])
    start = 0
    repeat = True
    for i in range(start,maxLimit):
        frontier = [initialNode]
        explored = {initialState.giveMeTuple() : 0}
        repeat,lastNode = dfsTillLimit(frontier,explored,maze,i,allStates)
        statesVisited += len(explored)
        if not repeat: break
    if repeat:
        print(-1)
        lastNode = None
    return lastNode, statesVisited

def findPath(lastNode):
    result = []
    while lastNode != None:
        result.append(lastNode.state.person)
        lastNode = lastNode.parent
    return str(result[::-1])
        

if __name__ == "__main__":
    maze, initialCord, goalCord, bagCapacity, balls, ballsTarget = getInput()

    initialState = State(initialCord,balls,ballsTarget)
    initialNode = Node(initialState)
    goalState = State(goalCord)
    allStates = [0]

    t1 = time.time()
    lastNode,statesVisited = IDS(initialCord,goalCord,maze,allStates)
    t2 = time.time()

    if lastNode != None: print("path is {}".format(findPath(lastNode)))
    print("distinct states visited are {}".format(statesVisited))
    print("all states are {}".format(allStates[0]))
    print("time = {}".format(t2 - t1))

requires 92 moves
path is [[1, 0], [1, 1], [1, 2], [1, 3], [2, 3], [3, 3], [3, 4], [3, 5], [3, 6], [3, 7], [3, 8], [3, 9], [4, 9], [5, 9], [5, 10], [6, 10], [7, 10], [8, 10], [9, 10], [10, 10], [11, 10], [12, 10], [12, 9], [12, 8], [12, 7], [13, 7], [14, 7], [15, 7], [15, 6], [15, 5], [15, 4], [16, 4], [17, 4], [18, 4], [19, 4], [20, 4], [20, 5], [20, 6], [20, 7], [20, 8], [20, 9], [20, 10], [20, 11], [21, 11], [22, 11], [22, 12], [22, 13], [22, 14], [22, 15], [22, 16], [22, 17], [21, 17], [20, 17], [19, 17], [18, 17], [18, 18], [18, 19], [18, 20], [17, 20], [16, 20], [16, 21], [16, 22], [16, 23], [15, 23], [14, 23], [13, 23], [13, 24], [13, 25], [14, 25], [15, 25], [16, 25], [17, 25], [18, 25], [19, 25], [20, 25], [21, 25], [22, 25], [22, 24], [22, 23], [22, 22], [23, 22], [24, 22], [25, 22], [26, 22], [27, 22], [28, 22], [28, 23], [28, 24], [28, 25], [28, 26], [28, 27], [28, 28], [28, 29]]
distinct states visited are 25522
all states are 229199
time = 1.8181214332580566


<strong>as you can see it is much slower than BFS or A* search which we will see further in the report.
this much time is used because when we reach a limit we throw away every thing and start over for a new limit hich we can avoid and make it faster but it is not the point of this assignment.
and also another point, expanded states and seen states in IDS is much more than BFS. but it is not just about the time.
if we go back to the BFS results we can see that for every test, the states that we see and the states that we expand are alot bigger in IDS.
but just like before expanded states are much less than seen states.</strong>


<h1> A* </h1>

<strong>تخمین را به این صورت در نظر گرفتم که در بدترین حالت از استیت کنونی تا استیت پایانی باید به اندازه اختلاف عرضی و اختلاف طولی حرکت داشته باشیم به طوری که همه ی توپ ها را هم در راهمان هست و برداریم (فرض بر آن است که هدف توپ ها در مسیرمان هست) 
</strong> <br/>
  <strong>      heuristic = abs(state.person[0] - goalCord[0]) + abs(state.person[1] - goalCord[1]) + len(currNode.state.balls)
</strong>

In [33]:
import time
import heapq
class Node:
    def __init__(self,state,heuristic = 0,parent = None,actions = 0, ballsTaken = 0):
        self.parent = parent
        self.state = state
        self.actions = actions
        self.ballsTaken = ballsTaken
        self.heuristic = heuristic
    
    def __lt__(self,other):
        return self.heuristic < other.heuristic

class State:
    def __init__(self,person,balls = [],ballsTarget = [],ballsCarryTarget = []):
        self.person = person
        self.balls = balls
        self.ballsTarget = ballsTarget
        self.ballsCarryTarget = ballsCarryTarget

    def giveMeTuple(self):
        return (tuple(self.person),tuple(self.balls),tuple(self.ballsCarryTarget))
    
    def __eq__(self,prev):
        return self.person == prev.person and self.balls == prev.balls and self.ballsCarryTarget == prev.ballsCarryTarget

def getInput():
    file = open('Tests/4.txt','r')
    lines = file.readlines()
    file.close()
    rowcol = list(map(int,lines[0].split()))
    startCords = list(map(int,lines[1].split()))
    endCords = list(map(int,lines[2].split()))
    bagCapacity = int(lines[3])
    numOfBalls = int(lines[4])
    startBallsCords = []
    endBallCords = []
    for n in range(numOfBalls):
        startEndCordBall = list(map(int,lines[n + 5].split()))
        startBallsCords.append(tuple(startEndCordBall[:2]))
        endBallCords.append(tuple(startEndCordBall[2:]))
    maze = []
    for i in range(rowcol[0]):
        maze.append(lines[5 + numOfBalls  + i].split())
    return maze, startCords, endCords, bagCapacity, startBallsCords, endBallCords

def addToFronterExplored(state,explored,frontier,goalCord,currNode,allStates,plusWhat = 0):
    allStates[0] += 1
    if state.giveMeTuple() not in explored:
        heuristic = abs(state.person[0] - goalCord[0]) + abs(state.person[1] - goalCord[1]) + len(currNode.state.balls)
        node = Node(state,heuristic + currNode.actions + 1,currNode,currNode.actions + 1,currNode.ballsTaken + plusWhat)
        heapq.heappush(frontier,node)

def moveRight(maze,currNode,explored,frontier,goalCord,allStates):
    if currNode.state.person[1] < len(maze[0]) - 1:
        if maze[currNode.state.person[0]][currNode.state.person[1] + 1] != '*':
            state = State([currNode.state.person[0] , currNode.state.person[1] + 1],currNode.state.balls[:],currNode.state.ballsTarget[:],currNode.state.ballsCarryTarget[:])           
            addToFronterExplored(state,explored,frontier,goalCord,currNode,allStates)

def moveLeft(maze,currNode,explored,frontier,goalCord,allStates):
    if currNode.state.person[1] > 0:
        if maze[currNode.state.person[0]][currNode.state.person[1] - 1] != '*':
            state = State([currNode.state.person[0] , currNode.state.person[1] - 1],currNode.state.balls[:],currNode.state.ballsTarget[:],currNode.state.ballsCarryTarget[:])
            addToFronterExplored(state,explored,frontier,goalCord,currNode,allStates)

def moveUp(maze,currNode,explored,frontier,goalCord,allStates):
    if currNode.state.person[0] > 0:
        if maze[currNode.state.person[0] - 1][currNode.state.person[1]] != '*':
            state = State([currNode.state.person[0] - 1 , currNode.state.person[1]],currNode.state.balls[:],currNode.state.ballsTarget[:],currNode.state.ballsCarryTarget[:])
            addToFronterExplored(state,explored,frontier,goalCord,currNode,allStates)
                
def moveDown(maze,currNode,explored,frontier,goalCord,allStates):
    if currNode.state.person[0] < len(maze) - 1:
        if maze[currNode.state.person[0] + 1][currNode.state.person[1]] != '*':
            state = State([currNode.state.person[0] + 1 , currNode.state.person[1]],currNode.state.balls[:],currNode.state.ballsTarget[:],currNode.state.ballsCarryTarget[:])
            addToFronterExplored(state,explored,frontier,goalCord,currNode,allStates)


def pickUpBall(currNode,bagCapacity,explored,frontier,goalCord,allStates):
    if currNode.ballsTaken < bagCapacity:
        for ballIndex in range(len(currNode.state.balls)):
            if tuple(currNode.state.person) == currNode.state.balls[ballIndex]:
                state = State(currNode.state.person,currNode.state.balls[:],currNode.state.ballsTarget[:],currNode.state.ballsCarryTarget[:])
                state.ballsCarryTarget.append(state.ballsTarget.pop(ballIndex))
                state.balls.pop(ballIndex)
                addToFronterExplored(state,explored,frontier,goalCord,currNode,allStates,1)
                break

def putDownBall(currNode):    
    for targetBall in range(len(currNode.state.ballsCarryTarget)):
        if tuple(currNode.state.person) == currNode.state.ballsCarryTarget[targetBall]:
            currNode.state.ballsCarryTarget.pop(targetBall)
            currNode.ballsTaken -= 1
            break

def aStar(initialNode,initialState,maze,goalCord,allStates):
    frontier = [initialNode]
    explored = {initialState.giveMeTuple() : ""}
    while 1:
        if len(frontier) == 0:
            print(-1)
            return None,len(explored)

        currNode = heapq.heappop(frontier)
        explored[currNode.state.giveMeTuple()] = ""

        putDownBall(currNode)
        if currNode.state == goalState:
            print("requires {} moves".format(currNode.actions))
            return currNode, len(explored)

        moveUp(maze,currNode,explored,frontier,goalCord,allStates)
        moveLeft(maze,currNode,explored,frontier,goalCord,allStates)
        moveDown(maze,currNode,explored,frontier,goalCord,allStates)
        moveRight(maze,currNode,explored,frontier,goalCord,allStates)
        pickUpBall(currNode,bagCapacity,explored,frontier,goalCord,allStates)

def findPath(lastNode):
    result = []
    while lastNode != None:
        result.append(lastNode.state.person)
        lastNode = lastNode.parent
    return str(result[::-1])
        

if __name__ == "__main__":
    maze, initialCord, goalCord, bagCapacity, balls, ballsTarget = getInput()

    initialState = State(initialCord,balls,ballsTarget)
    initialNode = Node(initialState,abs(initialCord[0] - goalCord[0]) + abs(initialCord[1] - goalCord[1]))
    goalState = State(goalCord)
    allStates = [0]

    t1 = time.time()
    lastNode,statesVisited = aStar(initialNode,initialState,maze,goalCord,allStates)
    t2 = time.time()

    if lastNode != None: print("path is {}".format(findPath(lastNode)))
    print("distinct states visited are {}".format(statesVisited))
    print("all states are {}".format(allStates[0]))
    print("time = {}".format(t2 - t1))

requires 92 moves
path is [[1, 0], [1, 1], [1, 2], [1, 3], [2, 3], [3, 3], [3, 4], [3, 5], [3, 6], [3, 7], [3, 8], [3, 9], [4, 9], [5, 9], [5, 10], [6, 10], [7, 10], [8, 10], [9, 10], [10, 10], [11, 10], [12, 10], [12, 9], [12, 8], [12, 7], [13, 7], [14, 7], [15, 7], [15, 6], [15, 5], [15, 4], [16, 4], [17, 4], [18, 4], [19, 4], [20, 4], [20, 5], [20, 6], [20, 7], [20, 8], [20, 9], [20, 10], [20, 11], [21, 11], [22, 11], [22, 12], [22, 13], [22, 14], [22, 15], [22, 16], [22, 17], [21, 17], [20, 17], [19, 17], [18, 17], [18, 18], [18, 19], [18, 20], [17, 20], [16, 20], [16, 21], [16, 22], [16, 23], [15, 23], [14, 23], [13, 23], [13, 24], [13, 25], [14, 25], [15, 25], [16, 25], [17, 25], [18, 25], [19, 25], [20, 25], [21, 25], [22, 25], [22, 24], [22, 23], [22, 22], [23, 22], [24, 22], [25, 22], [26, 22], [27, 22], [28, 22], [28, 23], [28, 24], [28, 25], [28, 26], [28, 27], [28, 28], [28, 29]]
distinct states visited are 457
all states are 1364
time = 0.00897359848022461


<h1>Weighted A*</h1>

<strong>در این مرحله باید به تخمین خود یک ضریب بدهیم به این منظور که لزوما بهینه ترین را نمیخواهیم بلکه هرچه زودتر جواب میخواهیم (هرچقدر به بهینه نزدیکتر باشد بهتر است)
</strong>

In [19]:
import time
import heapq
WEIGHT1 = 7
WEIGHT2 = 6
class Node:
    def __init__(self,state,heuristic = 0,parent = None,actions = 0, ballsTaken = 0):
        self.parent = parent
        self.state = state
        self.actions = actions
        self.ballsTaken = ballsTaken
        self.heuristic = heuristic
    
    def __lt__(self,other):
        return self.heuristic < other.heuristic

class State:
    def __init__(self,person,balls = [],ballsTarget = [],ballsCarryTarget = []):
        self.person = person
        self.balls = balls
        self.ballsTarget = ballsTarget
        self.ballsCarryTarget = ballsCarryTarget

    def giveMeTuple(self):
        return (tuple(self.person),tuple(self.balls),tuple(self.ballsCarryTarget))
    
    def __eq__(self,prev):
        return self.person == prev.person and self.balls == prev.balls and self.ballsCarryTarget == prev.ballsCarryTarget

def getInput():
    file = open('Tests/4.txt','r')
    lines = file.readlines()
    file.close()
    rowcol = list(map(int,lines[0].split()))
    startCords = list(map(int,lines[1].split()))
    endCords = list(map(int,lines[2].split()))
    bagCapacity = int(lines[3])
    numOfBalls = int(lines[4])
    startBallsCords = []
    endBallCords = []
    for n in range(numOfBalls):
        startEndCordBall = list(map(int,lines[n + 5].split()))
        startBallsCords.append(tuple(startEndCordBall[:2]))
        endBallCords.append(tuple(startEndCordBall[2:]))
    maze = []
    for i in range(rowcol[0]):
        maze.append(lines[5 + numOfBalls  + i].split())
    return maze, startCords, endCords, bagCapacity, startBallsCords, endBallCords

def addToFronterExplored(state,explored,frontier,goalCord,currNode,allStates,plusWhat = 0):
    allStates[0] += 1
    if state.giveMeTuple() not in explored:
        heuristic = abs(state.person[0] - goalCord[0]) + abs(state.person[1] - goalCord[1]) + len(currNode.state.balls)
        node = Node(state,heuristic * WEIGHT2 + currNode.actions + 1,currNode,currNode.actions + 1,currNode.ballsTaken + plusWhat)
        heapq.heappush(frontier,node)

def moveRight(maze,currNode,explored,frontier,goalCord,allStates):
    if currNode.state.person[1] < len(maze[0]) - 1:
        if maze[currNode.state.person[0]][currNode.state.person[1] + 1] != '*':
            state = State([currNode.state.person[0] , currNode.state.person[1] + 1],currNode.state.balls[:],currNode.state.ballsTarget[:],currNode.state.ballsCarryTarget[:])           
            addToFronterExplored(state,explored,frontier,goalCord,currNode,allStates)

def moveLeft(maze,currNode,explored,frontier,goalCord,allStates):
    if currNode.state.person[1] > 0:
        if maze[currNode.state.person[0]][currNode.state.person[1] - 1] != '*':
            state = State([currNode.state.person[0] , currNode.state.person[1] - 1],currNode.state.balls[:],currNode.state.ballsTarget[:],currNode.state.ballsCarryTarget[:])
            addToFronterExplored(state,explored,frontier,goalCord,currNode,allStates)

def moveUp(maze,currNode,explored,frontier,goalCord,allStates):
    if currNode.state.person[0] > 0:
        if maze[currNode.state.person[0] - 1][currNode.state.person[1]] != '*':
            state = State([currNode.state.person[0] - 1 , currNode.state.person[1]],currNode.state.balls[:],currNode.state.ballsTarget[:],currNode.state.ballsCarryTarget[:])
            addToFronterExplored(state,explored,frontier,goalCord,currNode,allStates)
                
def moveDown(maze,currNode,explored,frontier,goalCord,allStates):
    if currNode.state.person[0] < len(maze) - 1:
        if maze[currNode.state.person[0] + 1][currNode.state.person[1]] != '*':
            state = State([currNode.state.person[0] + 1 , currNode.state.person[1]],currNode.state.balls[:],currNode.state.ballsTarget[:],currNode.state.ballsCarryTarget[:])
            addToFronterExplored(state,explored,frontier,goalCord,currNode,allStates)


def pickUpBall(currNode,bagCapacity,explored,frontier,goalCord,allStates):
    if currNode.ballsTaken < bagCapacity:
        for ballIndex in range(len(currNode.state.balls)):
            if tuple(currNode.state.person) == currNode.state.balls[ballIndex]:
                state = State(currNode.state.person,currNode.state.balls[:],currNode.state.ballsTarget[:],currNode.state.ballsCarryTarget[:])
                state.ballsCarryTarget.append(state.ballsTarget.pop(ballIndex))
                state.balls.pop(ballIndex)
                addToFronterExplored(state,explored,frontier,goalCord,currNode,allStates,1)
                break

def putDownBall(currNode):    
    for targetBall in range(len(currNode.state.ballsCarryTarget)):
        if tuple(currNode.state.person) == currNode.state.ballsCarryTarget[targetBall]:
            currNode.state.ballsCarryTarget.pop(targetBall)
            currNode.ballsTaken -= 1
            break

def aStar(initialNode,initialState,maze,goalCord,allStates):
    frontier = [initialNode]
    explored = {initialState.giveMeTuple() : ""}
    while 1:
        if len(frontier) == 0:
            print(-1)
            return None,len(explored)

        currNode = heapq.heappop(frontier)
        explored[currNode.state.giveMeTuple()] = ""

        putDownBall(currNode)
        if currNode.state == goalState:
            print("requires {} moves".format(currNode.actions))
            return currNode, len(explored)

        moveUp(maze,currNode,explored,frontier,goalCord,allStates)
        moveLeft(maze,currNode,explored,frontier,goalCord,allStates)
        moveDown(maze,currNode,explored,frontier,goalCord,allStates)
        moveRight(maze,currNode,explored,frontier,goalCord,allStates)
        pickUpBall(currNode,bagCapacity,explored,frontier,goalCord,allStates)

def findPath(lastNode):
    result = []
    while lastNode != None:
        result.append(lastNode.state.person)
        lastNode = lastNode.parent
    return str(result[::-1])
        

if __name__ == "__main__":
    maze, initialCord, goalCord, bagCapacity, balls, ballsTarget = getInput()

    initialState = State(initialCord,balls,ballsTarget)
    initialNode = Node(initialState,abs(initialCord[0] - goalCord[0]) + abs(initialCord[1] - goalCord[1]))
    goalState = State(goalCord)
    allStates = [0]

    t1 = time.time()
    lastNode,statesVisited = aStar(initialNode,initialState,maze,goalCord,allStates)
    t2 = time.time()

    if lastNode != None: print("Path is {}".format(findPath(lastNode)))
    print("States visited are {}".format(statesVisited))
    print("All states are {}".format(allStates[0]))
    print("Time = {}".format(t2 - t1))

requires 92 moves
Path is [[1, 0], [1, 1], [1, 2], [1, 3], [2, 3], [3, 3], [4, 3], [5, 3], [5, 4], [5, 5], [5, 6], [5, 7], [5, 8], [5, 9], [5, 10], [6, 10], [7, 10], [8, 10], [9, 10], [10, 10], [11, 10], [12, 10], [12, 9], [12, 8], [12, 7], [13, 7], [14, 7], [15, 7], [15, 6], [15, 5], [15, 4], [16, 4], [17, 4], [18, 4], [19, 4], [20, 4], [20, 5], [20, 6], [20, 7], [20, 8], [20, 9], [20, 10], [20, 11], [21, 11], [22, 11], [22, 12], [22, 13], [22, 14], [22, 15], [22, 16], [22, 17], [21, 17], [20, 17], [19, 17], [18, 17], [18, 18], [18, 19], [18, 20], [17, 20], [16, 20], [16, 21], [16, 22], [16, 23], [15, 23], [14, 23], [13, 23], [13, 24], [13, 25], [14, 25], [15, 25], [16, 25], [17, 25], [18, 25], [19, 25], [20, 25], [21, 25], [22, 25], [22, 24], [22, 23], [22, 22], [23, 22], [24, 22], [25, 22], [26, 22], [27, 22], [28, 22], [28, 23], [28, 24], [28, 25], [28, 26], [28, 27], [28, 28], [28, 29]]
States visited are 392
All states are 1184
Time = 0.0053310394287109375


<strong> Test 1 result </strong>
<table>
    <tr>
        <th> الگوریتم </th>
        <th> فاصله جواب </th>
        <th> تعداد استیت دیده شده </th>
        <th> تعداد استیت مجزا</th>
        <th> میانگین زمانی </th>
    </tr>
    <tr> 
        <th> BFS </th>
        <th> 20 </th>
        <th> 648 </th>
        <th> 267 </th>
        <th> 0.004s </th>
    </tr>
    <tr> 
        <th> IDS </th>
        <th> 20 </th>
        <th> 7872 </th>
        <th> 2281 </th>
        <th> 0.02s </th>
    </tr>
    <tr> 
        <th> A* </th>
        <th> 20 </th>
        <th> 1010 </th>
        <th> 197 </th>
        <th> 0.004s </th>
    </tr>
    <tr> 
        <th> Weighted A* (a = 7) </th>
        <th> 24 </th>
        <th> 612 </th>
        <th> 128 </th>
        <th> 0.003s </th>
    </tr>
    <tr> 
        <th> Weighted A* (a = 6) </th>
        <th> 24 </th>
        <th> 585 </th>
        <th> 124 </th>
        <th> 0.002s </th>
    </tr>
    </table>

<strong> Test 2 result </strong>
<table>
    <tr>
        <th> الگوریتم </th>
        <th> فاصله جواب </th>
        <th> تعداد استیت دیده شده </th>
        <th> تعداد استیت مجزا</th>
        <th> میانگین زمانی </th>
    </tr>
    <tr> 
        <th> BFS </th>
        <th> 48 </th>
        <th> 9022 </th>
        <th> 2304 </th>
        <th> 0.004s </th>
    </tr>
    <tr> 
        <th> IDS </th>
        <th> 48 </th>
        <th> 1544253 </th>
        <th> 57720 </th>
        <th> 4s </th>
    <tr> 
        <th> A* </th>
        <th> 48 </th>
        <th> 1444 </th>
        <th> 233 </th>
        <th> 0.005s </th>
    </tr>
    <tr> 
        <th> Weighted A* (a = 7)</th>
        <th> 48 </th>
        <th> 168 </th>
        <th> 49 </th>
        <th> 0.0008s </th>
    </tr>
    <tr> 
        <th> Weighted A* (a = 6) </th>
        <th> 48 </th>
        <th> 168 </th>
        <th> 49 </th>
        <th> 0.001s </th>
    </tr>
    </table>

<strong> Test 3 result </strong>
<table>
    <tr>
        <th> الگوریتم </th>
        <th> فاصله جواب </th>
        <th> تعداد استیت دیده شده </th>
        <th> تعداد استیت مجزا</th>
        <th> میانگین زمانی </th>
    </tr>
    <tr> 
        <th> BFS </th>
        <th> 68 </th>
        <th> 659505 </th>
        <th> 235581 </th>
        <th> 11s </th>
    </tr>
    <tr> 
        <th> IDS </th>
        <th> 68 </th>
        <th> 14463347 </th>
        <th> 2288485 </th>
        <th> 50s </th>
    <tr> 
        <th> A* </th>
        <th> 68 </th>
        <th> 476320 </th>
        <th> 65322 </th>
        <th> 4s </th>
    </tr>
    <tr> 
        <th> Weighted A* (a = 7) </th>
        <th> 88 </th>
        <th> 57032 </th>
        <th> 9142 </th>
        <th> 0.2 </th>
    </tr>
    <tr> 
        <th> Weighted A* (a = 6) </th>
        <th> 88 </th>
        <th> 159319 </th>
        <th> 22856 </th>
        <th> 0.9s </th>
    </tr>
    </table>

<strong> Test 4 result </strong>
<table>
    <tr>
        <th> الگوریتم </th>
        <th> فاصله جواب </th>
        <th> تعداد استیت دیده شده </th>
        <th> تعداد استیت مجزا</th>
        <th> میانگین زمانی </th>
    </tr>
    <tr> 
        <th> BFS </th>
        <th> 92 </th>
        <th> 1019 </th>
        <th> 457 </th>
        <th> 0.003s </th>
    </tr>
    <tr> 
        <th> IDS </th>
        <th> 92 </th>
        <th> 229199 </th>
        <th> 25522 </th>
        <th> 0.06s </th>
    </tr>
    <tr> 
        <th> A* </th>
        <th> 92 </th>
        <th> 1364 </th>
        <th> 457 </th>
        <th> 0.005s </th>
    </tr>
    <tr> 
        <th> Weighted A* (a = 7) </th>
        <th> 92 </th>
        <th> 1209 </th>
        <th> 395 </th>
        <th> 0.04 </th>
    </tr>
    <tr> 
        <th> Weighted A* (a = 6) </th>
        <th> 92 </th>
        <th> 1184 </th>
        <th> 392 </th>
        <th> 0.005 </th>
    </tr>
    </table>